# Billboard Dataset Cleaning & Reshaping
This notebook loads, cleans, reshapes, and analyzes the Billboard dataset.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('billboard.csv', encoding='unicode_escape')
df.head()

## Observing the dataset
The dataset is in wide format, with each week stored as a separate column. Many week columns contain empty strings or inconsistent values.

In [ ]:
df.columns

## Melt the dataset into long format

In [ ]:
week_cols = [c for c in df.columns if 'week' in c]

df_melt = df.melt(
    id_vars=['year','artist.inverted','track','time','genre','date.entered','date.peaked'],
    value_vars=week_cols,
    var_name='Week',
    value_name='Rank'
)
df_melt.head()

## Clean Week column

In [ ]:
df_melt['Week'] = (
    df_melt['Week']
    .str.replace('x','', regex=False)
    .str.replace('.week','', regex=False)
    .str.replace('st','', regex=False)
    .str.replace('nd','', regex=False)
    .str.replace('rd','', regex=False)
    .str.replace('th','', regex=False)
)

df_melt['Week'] = pd.to_numeric(df_melt['Week'], errors='coerce')
df_melt.head()

## Compute the date at which the song had the given rank

In [ ]:
df_melt['date.entered'] = pd.to_datetime(df_melt['date.entered'])

df_melt['Rank_Date'] = df_melt['date.entered'] + pd.to_timedelta(df_melt['Week'] - 1, unit='W')
df_melt.head()

## Keep necessary columns

In [ ]:
df_clean = df_melt[['artist.inverted','track','genre','Week','Rank','Rank_Date']]
df_clean.head()

## Rename columns

In [ ]:
df_clean = df_clean.rename(columns={
    'artist.inverted': 'Artist',
    'track': 'Track',
    'genre': 'Genre',
    'Week': 'Week_Number',
    'Rank': 'Chart_Rank',
    'Rank_Date': 'Date_of_Rank'
})

df_clean.head()

## Clean NaN values

In [ ]:
df_clean['Chart_Rank'] = pd.to_numeric(df_clean['Chart_Rank'], errors='coerce')
df_clean = df_clean.dropna(subset=['Chart_Rank'])
df_clean.head()